In [36]:

import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess


In [37]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/robloxParentsComments.csv?token=GHSAT0AAAAAAB6ZNJNO2R3ZYCUN5JM76S62ZACGBVA')

In [22]:
df.head()

,url,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/RobloxParents/comment...,dianndianna,2021-05-06,1.620311e+09,4,I agree. I have been frustrated since download...,gx5jdbv,NaN
1,https://www.reddit.com/r/RobloxParents/comment...,cutenfuzzie,2021-05-06,1.620337e+09,3,I also agree. I am regretting ever letting my ...,gx79ppg,NaN
2,https://i.redd.it/wahei45uohz41.jpg,Deathly_banana,2020-06-11,1.591900e+09,3,Hehehe secy,ftq05wx,NaN
3,https://i.redd.it/wahei45uohz41.jpg,Afimiliarface,2020-07-01,1.593626e+09,2,I want to report this too what’s the game call...,fwlsxns,NaN
4,https://i.redd.it/wahei45uohz41.jpg,StupidSam1,2020-05-29,1.590754e+09,1,"Report it, these games are from around 2014 an...",fs6xz8e,NaN


In [23]:
df['comment']

0      I agree. I have been frustrated since download...
1      I also agree. I am regretting ever letting my ...
2                                            Hehehe secy
3      I want to report this too what’s the game call...
4      Report it, these games are from around 2014 an...
                             ...                        
166    No one really plays Murder Mystery 2 for the g...
167    Murder Mystery 2 is one of my favorite games o...
168                                Buy another computer.
169    Both games are in PC, Xbox, or Mobile. You can...
170    Have them take turns. Child 1 gets computer 2p...
Name: comment, Length: 171, dtype: object

In [24]:
## Key Words

In [26]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [27]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
print(bi_grams_list)

[('agree', 'frustrated'), ('frustrated', 'since'), ('since', 'downloading'), ('downloading', 'Roblox'), ('Roblox', 'son'), ('son', 'almost'), ('almost', '9'), ('9', 'months'), ('months', 'ago'), ('ago', 'would'), ('would', 'like'), ('like', 'able'), ('able', 'block'), ('block', 'games'), ('games', 'block'), ('block', 'content'), ('content', 'countries'), ('countries', 'due'), ('due', 'less'), ('less', 'monitoring'), ('monitoring', 'even'), ('even', 'make'), ('make', 'curated'), ('curated', 'list'), ('list', 'games'), ('games', 'think'), ('think', 'okay'), ('okay', 'games'), ('games', 'shown'), ('shown', 'allowed'), ('allowed', 'turn'), ('turn', 'account'), ('account', 'restrictions'), ('restrictions', 'basically'), ('basically', 'struggles'), ('struggles', 'find'), ('find', 'much'), ('much', 'anything'), ('anything', 'play'), ('play', 'gets'), ('gets', 'upset'), ('upset', 'going'), ('going', 'game'), ('game', 'game'), ('game', 'find'), ('find', 'let'), ('let', 'play'), ('play', 'really

In [28]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

print(bi_grams_list1)
freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

['agree frustrated', 'frustrated since', 'since downloading', 'downloading Roblox', 'Roblox son', 'son almost', 'almost 9', '9 months', 'months ago', 'ago would', 'would like', 'like able', 'able block', 'block games', 'games block', 'block content', 'content countries', 'countries due', 'due less', 'less monitoring', 'monitoring even', 'even make', 'make curated', 'curated list', 'list games', 'games think', 'think okay', 'okay games', 'games shown', 'shown allowed', 'allowed turn', 'turn account', 'account restrictions', 'restrictions basically', 'basically struggles', 'struggles find', 'find much', 'much anything', 'anything play', 'play gets', 'gets upset', 'upset going', 'going game', 'game game', 'game find', 'find let', 'let play', 'play really', 'really cater', 'cater kids', 'kids seem', 'seem driven', 'driven kids', 'kids playing', 'playing platform', 'platform wish', 'wish would', 'would take', 'take parental', 'parental control', 'control settings', 'settings seriously', 'se

In [45]:
words = text.split()
words = [word for word in words if word.lower() not in stop_words]
word_counts = Counter(words)

for word, count in word_counts.most_common(10):
    print(f"{word}: {count}")

games: 38
game: 33
get: 30
Roblox: 26
Robux: 26
would: 21
play: 21
roblox: 20
account: 18
like: 17


In [29]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                                             term  rank
229                                   beast mode   8.0
75                          account restrictions   8.0
296                                        ca nt   6.0
1482                           parental controls   6.0
960                https wwwrobloxcommymoneyaspx   5.0
2395  wwwrobloxcommymoneyaspx mytransactions_tab   5.0
283                                    buy robux   5.0


In [30]:
df_ques = df['comment'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['comment'])

4      Report it, these games are from around 2014 an...
9                                                      ?
13                                    clickbaited games?
83                 Can i offer Candy Set and logchopper?
94     I don’t really understand what happened, the v...
95     It was like a clip before the game opened (aft...
97     How long did it take to retain access to the r...
106    do the settings described [here](https://en.he...
139    Well your son has to learn.\nAl, he needs is a...
147              Hmm, have you check your local Walmart?
Name: comment, dtype: object


In [33]:
df['text_processed'] = df['comment'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    i agree i have been frustrated since downloadi...
1    i also agree i am regretting ever letting my c...
2                                          hehehe secy
3    i want to report this too what’s the game call...
4    report it these games are from around 2014 and...
Name: text_processed, dtype: object

In [34]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.comment.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['agree', 'frustrated', 'since', 'downloading', 'roblox', 'son', 'almost', 'months', 'ago', 'would', 'like', 'able', 'block', 'games', 'block', 'content', 'countries', 'due', 'less', 'monitoring', 'even', 'make', 'curated', 'list', 'games', 'think', 'okay', 'games', 'shown', 'allowed']


In [55]:
#os.makedirs('./results')
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#os.makedirs('./results')
pyLDAvis.save_html(LDAvis_prepared, './results'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
